Move all image files from one S3 bucket to another S3 bucket, but only if the image has no transparent pixels.

Objective: Write a Python script that uses the Boto3 library to accomplish the following:

* List all the image files in a given S3 bucket
* Check if each image file has transparent pixels
* If an image file has no transparent pixels, copy it to a different S3 bucket
* If an image file has transparent pixels, log it in a separate file

Guidelines:

* Your script should take the name of the source and destination buckets as input
* You should use the Boto3 library to interact with S3
* You should use the Pillow library to check for transparent pixels in an image
* Your script should handle any errors that may occur during the opening of image file, copy process and anywhere else you deem necessary
* Your script should be well commented and easy to understand
* Your script should be executed from the command line

In [2]:
import boto3
from PIL import Image
import logging
import os

def move_images(src_folder, dst_bucket):
    # Connect to S3
    s3 = boto3.resource('s3')
    dst = s3.Bucket(dst_bucket)

    # Create a logger
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    # Loop through all files in the source folder
    for filename in os.listdir(src_folder):
        file_path = os.path.join(src_folder, filename)
        # Check if the file is an image file
        if filename.endswith('.png') or filename.endswith('.jpg') or filename.endswith('.jpeg'):
            try:
                # Open the image file
                img = Image.open(file_path)

                # Check if the image has no transparent pixels
                if not img.getbbox():
                    # Copy the image file to the destination bucket
                    dst.upload_file(file_path, filename)
                else:
                    # Log the image file that has transparent pixels
                    logger.info('{} has transparent pixels'.format(filename))
            except Exception as e:
                # Log the error
                logger.error('Error while processing {}: {}'.format(filename, e))


    import argparse

    # Parse the command line arguments
    parser = argparse.ArgumentParser(description='Move image files without transparent pixels from a local folder to an S3 bucket')
    parser.add_argument('src_folder', type=str, help='The path to the source folder')
    parser.add_argument('dst_bucket', type=str, help='The name of the destination S3 bucket')
    args = parser.parse_args()

    # Call the function to move the images
    move_images(args.src_folder, args.dst_bucket)


usage: ipykernel_launcher.py [-h] src_folder dst_bucket
ipykernel_launcher.py: error: the following arguments are required: dst_bucket


SystemExit: 2

c:\users\astro\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
